In [1]:
import pandas as pd 
import numpy as np

In [2]:
data=pd.read_csv("CompanyData.csv")

In [4]:
data.head()

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US
0,9.50,138,73,11,276,120,Bad,42,17,Yes,Yes
1,11.22,111,48,16,260,83,Good,65,10,Yes,Yes
2,10.06,113,35,10,269,80,Medium,59,12,Yes,Yes
3,7.40,117,100,4,466,97,Medium,55,14,Yes,Yes
4,4.15,141,64,3,340,128,Bad,38,13,Yes,No


In [8]:
data1=pd.get_dummies(data,drop_first=True)

In [9]:
data1

,Sales,CompPrice,Income,Advertising,Population,Price,Age,Education,ShelveLoc_Good,ShelveLoc_Medium,Urban_Yes,US_Yes
0,9.50,138,73,11,276,120,42,17,0,0,1,1
1,11.22,111,48,16,260,83,65,10,1,0,1,1
2,10.06,113,35,10,269,80,59,12,0,1,1,1
3,7.40,117,100,4,466,97,55,14,0,1,1,1
4,4.15,141,64,3,340,128,38,13,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
395,12.57,138,108,17,203,128,33,14,1,0,1,1
396,6.14,139,23,3,37,120,55,11,0,1,0,1
397,7.41,162,26,12,368,159,40,18,0,1,1,1
398,5.94,100,79,7,284,95,50,12,0,0,1,1


In [10]:
labels=["bad","average","good"]
bins=[0,5,10,17]

In [12]:
data1['Sales']=pd.cut(data1["Sales"],bins=bins,labels=labels)

In [13]:
data1.isnull().sum()

Sales               1
CompPrice           0
Income              0
Advertising         0
Population          0
Price               0
Age                 0
Education           0
ShelveLoc_Good      0
ShelveLoc_Medium    0
Urban_Yes           0
US_Yes              0
dtype: int64

In [15]:
colnames=list(data1.columns)

In [16]:
data1.drop([174],inplace=True,axis=0)

In [17]:
predictors=colnames[1:12]

In [18]:
target=colnames[0]

In [19]:
data1['in_train']=np.random.uniform(0,1,len(data1))<=0.75

In [22]:
help(np.random.randint)

Help on built-in function randint:

randint(...) method of numpy.random.mtrand.RandomState instance
    randint(low, high=None, size=None, dtype=int)
    
    Return random integers from `low` (inclusive) to `high` (exclusive).
    
    Return random integers from the "discrete uniform" distribution of
    the specified dtype in the "half-open" interval [`low`, `high`). If
    `high` is None (the default), then results are from [0, `low`).
    
    .. note::
        New code should use the ``integers`` method of a ``default_rng()``
        instance instead; please see the :ref:`random-quick-start`.
    
    Parameters
    ----------
    low : int or array-like of ints
        Lowest (signed) integers to be drawn from the distribution (unless
        ``high=None``, in which case this parameter is one above the
        *highest* such integer).
    high : int or array-like of ints, optional
        If provided, one above the largest (signed) integer to be drawn
        from the distributi

In [23]:
true,false=data1[data1['in_train']==True],data1[data1['in_train']==False]

In [24]:
from sklearn.model_selection import train_test_split

In [25]:
train,test=train_test_split(data1,test_size=0.2)

In [26]:
from sklearn.tree import DecisionTreeClassifier as DTC

In [30]:
model=DTC(criterion='entropy')
model.fit(train[predictors],train[target])
preds=model.predict(train[predictors])

In [31]:
pd.crosstab(preds,train[target])

Sales,bad,average,good
row_0,,,
average,0,193,0
bad,67,0,0
good,0,0,59


In [32]:
np.mean(train['Sales']==preds)

1.0

# Testing

In [33]:
pred_test=model.predict(test[predictors])

In [35]:
pd.crosstab(test['Sales'],test[target])

Sales,bad,average,good
Sales,,,
bad,9,0,0
average,0,52,0
good,0,0,19


In [36]:
np.mean(test['Sales']==model.predict(test[predictors]))

0.675